# Improved Fashion MNIST accuracy using CNN

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from functools import partial

## Load Fashion MNIST dataset

In [9]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(train_images, train_labels), (test_images, test_labels) = fashion_mnist

# Scale pixel intensities (immediate 10x improvement to accuracy)

train_images = train_images / 255.0
test_images = test_images / 255.0

## Create Model

In [10]:
DefaultConv2D = partial(tf.keras.layers.Conv2D,
                        kernel_size=3, padding="same", activation='relu', kernel_initializer='he_normal')

model = tf.keras.Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[28, 28, 1]), # Images are 28 x 28 grayscale
    tf.keras.layers.MaxPooling2D(pool_size=2), # Reduce dimensions by 1/2
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Flatten(), # B/c dense layer expects 1D array of inputs

    # Fully connected network
    tf.keras.layers.Dense(units=128, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(0.5), # Reduce overfitting
    tf.keras.layers.Dense(units=64, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=10, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.001)

history = model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

## Train Model

In [12]:
model.fit(train_images, train_labels, epochs=30, validation_split = 0.2)

Epoch 1/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 19s 9ms/step - accuracy: 0.6175 - loss: 1.0960 - val_accuracy: 0.8641 - val_loss: 0.3737
Epoch 2/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - accuracy: 0.8459 - loss: 0.4545 - val_accuracy: 0.8858 - val_loss: 0.3126
Epoch 3/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - accuracy: 0.8772 - loss: 0.3700 - val_accuracy: 0.8957 - val_loss: 0.2918
Epoch 4/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.8907 - loss: 0.3259 - val_accuracy: 0.8987 - val_loss: 0.2747
Epoch 5/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - accuracy: 0.8984 - loss: 0.3026 - val_accuracy: 0.9077 - val_loss: 0.2459
Epoch 6/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.9079 - loss: 0.2675 - val_accuracy: 0.9000 - val_loss: 0.2946
Epoch 7/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - accuracy: 0.9159 - loss: 0.2523 - val_accuracy: 0.9052 - val_loss: 0.2594
Epoch 8/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.9184 - loss: 0

## Evaluate Model

In [13]:
model.evaluate(test_images, test_labels)

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9122 - loss: 0.4543


[0.45605841279029846, 0.911300003528595]

## Save Model

In [14]:
model.save('fashion_mnist_cnn.keras')